In [1]:
%cd C:\Case Western\HT correlation Updated 6-4
    
    
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
#from xgboost import XGBRegressor

C:\Case Western\HT correlation Updated 6-4


In [148]:
# Read all the .xls files. Files saves with .xlsx will not be read. 
path = os.getcwd()
files = os.listdir(path)
#files

files_xls = [f for f in files if f[-3:] == 'xls']
print(len(files_xls))
#files_xls

46


In [3]:
# Concatenate all the xl files into 1 dataframe. 
df_ALL = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(f, 'Sheet1')
    data['filename'] = os.path.basename(f)
    df_ALL = df_ALL.append(data)
    
df_ALL = df_ALL.iloc[2:,:]
df_ALL.rename(columns={'temperature.1': 'Temperature'}, inplace=True)
df_ALL.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Bd,Bo,Channel Geometry,Channel Material,Channel Orientation,Co,Cpf,Cpg,Critical Pressure,Cvf,...,kf,kg,mass velocity,number of channels,quality,temperature,Temperature,μf,μg,σ
2,0.14756,0.000166,R,S+L,H,5.841979,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.00742,25,298.15,285.49,10.852,0.010068
3,0.14756,0.000166,R,S+L,H,3.833296,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.0125,25,298.15,285.49,10.852,0.010068
4,0.14756,0.000166,R,S+L,H,2.850637,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.018,25,298.15,285.49,10.852,0.010068
5,0.14756,0.000166,R,S+L,H,2.179327,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.025,25,298.15,285.49,10.852,0.010068
6,0.14756,0.000166,R,S+L,H,1.860486,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.0303,25,298.15,285.49,10.852,0.010068


In [5]:
# Get all column names
df_ALL.columns

Index(['Bd', 'Bo', 'Channel Geometry', 'Channel Material',
       'Channel Orientation', 'Co', 'Cpf', 'Cpg', 'Critical Pressure', 'Cvf',
       'Cvg', 'Density-f', 'Density-g', 'Frf', 'Frfo', 'Frg', 'Frgo',
       'HT Characteristics', 'PF', 'PH', 'PR', 'Pef', 'Peg', 'Phase',
       'Pressure', 'Prf', 'Prg', 'Ref', 'Refo', 'Reg', 'Rego',
       'Roughness [e/Dh)', 'Suf', 'Sug', 'Wef', 'Wefo', 'Weg', 'Wego', 'Xtt',
       'aspect ratio', 'diameter', 'filename', 'fluid', 'h', 'heat flux',
       'hfg', 'kf', 'kg', 'mass velocity', 'number of channels', 'quality',
       'temperature', 'Temperature', 'μf', 'μg', 'σ'],
      dtype='object')

In [6]:
df_consol = df_ALL
df_consol = df_consol.loc[df_consol.index >1]
df_consol.head(3)

,Bd,Bo,Channel Geometry,Channel Material,Channel Orientation,Co,Cpf,Cpg,Critical Pressure,Cvf,...,kf,kg,mass velocity,number of channels,quality,temperature,Temperature,μf,μg,σ
2,0.14756,0.000166,R,S+L,H,5.841979,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.00742,25,298.15,285.49,10.852,0.010068
3,0.14756,0.000166,R,S+L,H,3.833296,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.0125,25,298.15,285.49,10.852,0.010068
4,0.14756,0.000166,R,S+L,H,2.850637,1.2644,0.88495,32,0.89685,...,0.072826,0.012773,280.7,multi,0.018,25,298.15,285.49,10.852,0.010068


In [7]:
# Write Consolidated data in .xlsx format.
import xlsxwriter as xlsw
xlsfile = 'Consol.xlsx'
writer = pd.ExcelWriter(xlsfile, engine='xlsxwriter')

df_consol.to_excel(writer, sheet_name='Sheet1',startrow=0, startcol=0, header=True ,index=True)
writer.save()

In [8]:
df_consol['SUgo'] = df_consol['Density-g']*df_consol['σ']*df_consol['diameter']/(df_consol['μg']**2)

#df_consol.fillna(0,inplace=True)
df_consol.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Bd,Bo,Channel Geometry,Channel Material,Channel Orientation,Co,Cpf,Cpg,Critical Pressure,Cvf,...,kg,mass velocity,number of channels,quality,temperature,Temperature,μf,μg,σ,SUgo
2,0.14756,0.000166,R,S+L,H,5.841979,1.2644,0.88495,32,0.89685,...,0.012773,280.7,multi,0.00742,25,298.15,285.49,10.852,0.010068,0.000528055
3,0.14756,0.000166,R,S+L,H,3.833296,1.2644,0.88495,32,0.89685,...,0.012773,280.7,multi,0.0125,25,298.15,285.49,10.852,0.010068,0.000528055
4,0.14756,0.000166,R,S+L,H,2.850637,1.2644,0.88495,32,0.89685,...,0.012773,280.7,multi,0.018,25,298.15,285.49,10.852,0.010068,0.000528055
5,0.14756,0.000166,R,S+L,H,2.179327,1.2644,0.88495,32,0.89685,...,0.012773,280.7,multi,0.025,25,298.15,285.49,10.852,0.010068,0.000528055
6,0.14756,0.000166,R,S+L,H,1.860486,1.2644,0.88495,32,0.89685,...,0.012773,280.7,multi,0.0303,25,298.15,285.49,10.852,0.010068,0.000528055


In [157]:
#cat_cols = ['Channel Geometry','Channel Material'] #,'HT Characteristics'
# 
#cols = ['Pressure','Critical Pressure','diameter','Temperature','Roughness [e/Dh)','mass velocity','quality','heat flux','Density-g','Density-f','hfg','Cvg','Cvf','Cpg','Cpf','μg','μf','σ']
cols = ['Reg','Ref','Rego','Refo','Sug','Suf','Wef','Wefo','Weg','Wego','Bo','Frf','Frfo','Frg','Frgo','Bd','Pef','Peg','heat flux','diameter','quality','mass velocity','filename'] 
#cols = ['Bd', 'Bo', 'Channel Geometry', 'Channel Material','Channel Orientation', 'Co', 'Cpf', 'Cpg', 'Critical Pressure', 'Cvf','Cvg', 'Density-f', 'Density-g', 'Frf', 'Frfo','Frg', 'Frgo','HT Characteristics', 'PF', 'PH', 'PR', 'Pef', 'Peg', 'Phase','Pressure', 'Prf', 'Prg', 'Ref', 'Refo', 'Reg', 'Rego','Roughness [e/Dh)', 'Suf', 'Sug', 'Wef', 'Wefo', 'Weg', 'Wego', 'Xtt','aspect ratio', 'diameter', 'fluid', 'h', 'heat flux', 'hfg', 'kf','kg', 'mass velocity', 'number of channels', 'quality', 'temperature','Temperature', 'μf', 'μg', 'σ'] 
#cols = ['diameter','Temperature','Reg', 'Ref','Rego', 'Refo','Sug','Bo','PH','PR', 'Xtt', 'Wefo','mass velocity','heat flux']
#cols = ['Reg','Ref','Bo','Sug','Suf','Frf','Frg','Wef','Weg']
df_consol2 = df_consol[cols]

#df_consol2.rename(columns={'Roughness [e/Dh)': 'Roughness'}, inplace=True)
#df_consol2['Roughness'].fillna(0,inplace=True)
#df_consol2['Roughness'] = df_consol2['Roughness'].replace({'Smooth': 0})


df_consol2.fillna(0,inplace=True)
df_consol2.head(3)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,Reg,Ref,Rego,Refo,Sug,Suf,Wef,Wefo,Weg,Wego,...,Frg,Frgo,Bd,Pef,Peg,heat flux,diameter,quality,mass velocity,filename
2,64.487540,327.911245,8691.043126,330.362535,528055.172777,56438.533421,1.905184,1.933775,0.007875,143.042308,...,3.894499,70736.532368,0.14756,1625.343806,48.485448,0.678852,0.336,0.00742,280.7,Agostini et al (2008).xls
3,108.638039,326.233003,8691.043126,330.362535,528055.172777,56438.533421,1.885732,1.933775,0.022350,143.042308,...,11.052583,70736.532368,0.14756,1617.025336,81.680338,0.678852,0.336,0.01250,280.7,Agostini et al (2008).xls
4,156.438776,324.416009,8691.043126,330.362535,528055.172777,56438.533421,1.864785,1.933775,0.046346,143.042308,...,22.918636,70736.532368,0.14756,1608.019119,117.619686,0.678852,0.336,0.01800,280.7,Agostini et al (2008).xls


In [14]:
df_consol2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Reg,15474.0,1.756621e+04,2.217045e+04,2.898024,3.777474e+03,1.035897e+04,2.342383e+04,3.597947e+05
Ref,15474.0,2.457443e+03,3.385386e+03,1.954273,6.667893e+02,1.445914e+03,2.835016e+03,5.524535e+04
Rego,15474.0,5.701313e+04,5.361824e+04,623.522699,2.266641e+04,3.909915e+04,6.965343e+04,4.109518e+05
Refo,15474.0,3.791217e+03,4.826576e+03,27.152694,1.217629e+03,2.594081e+03,4.294191e+03,5.527000e+04
Sug,15474.0,2.768325e+06,2.374190e+06,35115.699267,1.072748e+06,2.030903e+06,3.943501e+06,1.404366e+07
Suf,15474.0,4.264293e+05,4.284077e+05,29502.006211,2.254764e+05,2.867080e+05,5.541045e+05,3.136613e+06
Wef,15474.0,2.842624e+01,9.096730e+01,0.000032,1.755517e+00,7.626072e+00,2.528769e+01,3.139787e+03
Wefo,15474.0,5.649416e+01,1.694711e+02,0.006910,8.056836e+00,2.182412e+01,5.302905e+01,3.142589e+03
Weg,15474.0,2.046962e+02,5.393985e+02,0.000014,9.547535e+00,5.419154e+01,2.096394e+02,2.091888e+04
Wego,15474.0,1.768355e+03,2.772737e+03,1.373992,3.540643e+02,7.636462e+02,1.830846e+03,3.269692e+04


In [158]:
df_tar = df_consol[['h']]
df_tar.head(3)

,h
2,1640
3,1420
4,1320


In [159]:
X_1 = df_consol2

print(len(X_1.columns))
X_1.columns

23


Index(['Reg', 'Ref', 'Rego', 'Refo', 'Sug', 'Suf', 'Wef', 'Wefo', 'Weg',
       'Wego', 'Bo', 'Frf', 'Frfo', 'Frg', 'Frgo', 'Bd', 'Pef', 'Peg',
       'heat flux', 'diameter', 'quality', 'mass velocity', 'filename'],
      dtype='object')

In [160]:
xlsfile = 'X_1.xlsx'
writer = pd.ExcelWriter(xlsfile, engine='xlsxwriter')
dataset = X_1
dataset.to_excel(writer, sheet_name='Sheet1',startrow=0, startcol=0, header=True ,index=True)
writer.save()

In [161]:
df_Author = X_1['filename'].value_counts().to_frame()
df_Author.reset_index(inplace=True)
df_Author.head(2)

,index,filename
0,Ong (2010).xls,2504
1,Ducoulombier (2010).xls,1573


In [162]:
X_1.head(5)

,Reg,Ref,Rego,Refo,Sug,Suf,Wef,Wefo,Weg,Wego,...,Frg,Frgo,Bd,Pef,Peg,heat flux,diameter,quality,mass velocity,filename
2,64.487540,327.911245,8691.043126,330.362535,528055.172777,56438.533421,1.905184,1.933775,0.007875,143.042308,...,3.894499,70736.532368,0.14756,1625.343806,48.485448,0.678852,0.336,0.00742,280.7,Agostini et al (2008).xls
3,108.638039,326.233003,8691.043126,330.362535,528055.172777,56438.533421,1.885732,1.933775,0.022350,143.042308,...,11.052583,70736.532368,0.14756,1617.025336,81.680338,0.678852,0.336,0.01250,280.7,Agostini et al (2008).xls
4,156.438776,324.416009,8691.043126,330.362535,528055.172777,56438.533421,1.864785,1.933775,0.046346,143.042308,...,22.918636,70736.532368,0.14756,1608.019119,117.619686,0.678852,0.336,0.01800,280.7,Agostini et al (2008).xls
5,217.276078,322.103471,8691.043126,330.362535,528055.172777,56438.533421,1.838295,1.933775,0.089401,143.042308,...,44.210333,70736.532368,0.14756,1596.556661,163.360675,0.678852,0.336,0.02500,280.7,Agostini et al (2008).xls
6,263.338607,320.352550,8691.043126,330.362535,528055.172777,56438.533421,1.818363,1.933775,0.131326,143.042308,...,64.942503,70736.532368,0.14756,1587.877943,197.993139,0.678852,0.336,0.03030,280.7,Agostini et al (2008).xls


In [163]:
X_1['filename']=X_1['filename'].astype('category')

X_1['filename']=X_1['filename'].cat.codes
df_Author_num = X_1['filename'].value_counts().to_frame()
df_Author_num.reset_index(inplace=True)
df_Author_num.head(2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,filename
0,29,2504
1,13,1573


In [164]:
# Merge Author Name by Num

df_Author_map = df_Author.merge(df_Author_num, how='outer', left_index=True, right_index=True)
df_Author_map.head(3)

,index_x,filename_x,index_y,filename_y
0,Ong (2010).xls,2504,29,2504
1,Ducoulombier (2010).xls,1573,13,1573
2,Copetti et al (2011).xls,876,9,876


In [143]:
# Write the results of test dataset in excel file
import xlsxwriter as xlsw
xlsfile = 'df_Author_map.xlsx'
writer = pd.ExcelWriter(xlsfile, engine='xlsxwriter')

df_Author_map.to_excel(writer, sheet_name='Sheet1',startrow=0, startcol=0, header=True ,index=True)
writer.save()

In [165]:
# Split datasets into train and test. Random_state is an arbitrary number to reproduce the results. 
X = X_1
y = df_tar

X_train, X_test, y_train, y_test = train_test_split(X, y,shuffle=True,random_state=10)

scaler = StandardScaler().fit(X_train)

scaler.fit(X_train)

X_train1 = scaler.transform(X_train)
X_test1 = scaler.transform(X_test)
X_train1= X_train1[:,:-1]
X_test1 = X_test1[:,:-1]

In [92]:
# Drop the Filename columns
#X_train_tag = X_train1[:,-1]
#X_test_tag = X_test1[:,-1]



In [154]:
#df_tag = pd.DataFrame({'FileName_Tag':X_train_tag})
#df_tag['FileName_Tag'].value_counts()

In [155]:
#X_train['filename'].value_counts()

In [166]:
# ANN Model logging loss output with iterations.  

#sys.stdout=open("ANN1.txt","w") # To write the output in a text file. 

mlp = MLPRegressor(hidden_layer_sizes=(60,50,40,40,35,30,20,10),max_iter=30000,activation='relu',random_state=4)#,verbose=True)
mlp.fit(X_train1,y_train.values.ravel())

predictions = mlp.predict(X_test1)
predictions

actual = y_test.astype('float64').values.ravel()
actual


r2_score(actual, predictions)

0.4687648074989732

In [167]:
X_test['Predictions'] = pd.DataFrame(predictions)
X_test['Actual'] = y_test['h']
#X_test.head(5)
dataset = X_test.merge(df_Author_map, how='left', left_on=['filename'], right_on=['index_y'])
#dataset.head(3)
dataset = dataset.drop(['filename','filename_x','filename_y','index_y'],axis=1)
# Write the results of test dataset in excel file
import xlsxwriter as xlsw
xlsfile = 'Test_Results.xlsx'
writer = pd.ExcelWriter(xlsfile, engine='xlsxwriter')

dataset.to_excel(writer, sheet_name='Sheet1',startrow=0, startcol=0, header=True ,index=True)
writer.save()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [54]:
mlp = MLPRegressor(hidden_layer_sizes=(65,60,50,40,40,35,30,25,20,10),max_iter=30000,activation='relu',random_state=4)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions

actual = y_test.astype('float64').values.ravel()
actual

r2_score(actual, predictions)

0.4410215334033085

In [55]:
mlp = MLPRegressor(hidden_layer_sizes=(40,20,15,10),max_iter=30000,activation='relu',random_state=4)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions

actual = y_test.astype('float64').values.ravel()
actual

r2_score(actual, predictions)

0.4228864388954613

In [18]:
mlp = MLPRegressor(hidden_layer_sizes=(20,20,20,20,15,10),max_iter=30000,activation='relu',random_state=4)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions

actual = y_test.astype('float64').values.ravel()
actual

r2_score(actual, predictions)

0.5495743616972577

In [19]:
mlp = MLPRegressor(hidden_layer_sizes=(30,25,20,20,15,10),max_iter=30000,activation='relu',random_state=4)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions

actual = y_test.astype('float64').values.ravel()
actual

r2_score(actual, predictions)

0.53518015679004

In [20]:

actual = y_test.astype('float64').values.ravel()
actualmlp = MLPRegressor(hidden_layer_sizes=(40,30,25,20,20,15,10),max_iter=30000,activation='relu',random_state=4)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions


r2_score(actual, predictions)

0.53518015679004

In [21]:

actual = y_test.astype('float64').values.ravel()
actualmlp = MLPRegressor(hidden_layer_sizes=(1),max_iter=30000,activation='relu',random_state=5)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions


r2_score(actual, predictions)

0.53518015679004

In [22]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=5, random_state=0,n_estimators=150)
regr.fit(X_train, y_train.values.ravel())

pred_randfor = regr.predict(X_test)
r2_score(y_test.values.ravel(), pred_randfor)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.7511076572106128

In [23]:
regr.feature_importances_

array([3.07261230e-02, 4.40904105e-02, 4.14977091e-03, 4.07005799e-03,
       9.30583869e-04, 1.00876391e-03, 3.49199916e-02, 4.95665002e-03,
       4.15189739e-02, 7.91538193e-04, 6.18877401e-02, 3.55108595e-02,
       4.53993750e-02, 5.13228270e-02, 3.28730008e-01, 1.67626713e-04,
       5.54764586e-02, 5.63140393e-02, 1.49965815e-01, 5.21356133e-05,
       4.14833212e-02, 6.52693072e-03])

In [24]:
X_1.columns

Index(['Reg', 'Ref', 'Rego', 'Refo', 'Sug', 'Suf', 'Wef', 'Wefo', 'Weg',
       'Wego', 'Bo', 'Frf', 'Frfo', 'Frg', 'Frgo', 'Bd', 'Pef', 'Peg',
       'heat flux', 'diameter', 'quality', 'mass velocity'],
      dtype='object')

In [25]:
XGBModel = XGBRegressor(random_state=2)
XGBModel.fit(X_train, y_train.values.ravel())

predictions = XGBModel.predict(X_test)
predictions

actual = y_test.astype('float64').values.ravel()
actual

r2_score(actual, predictions)

[08:25:22] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.8807538676555988

In [26]:
# Running the selected ANN again to estimate MAE of test dataset
mlp = MLPRegressor(hidden_layer_sizes=(60,50,40,40,35,30,20,10),max_iter=30000,activation='relu',random_state=4)
mlp.fit(X_train,y_train.values.ravel())

predictions = mlp.predict(X_test)
predictions

actual = y_test.astype('float64').values.ravel()
actual

r2_score(actual, predictions)

0.4287336180091782

In [27]:
# MAE of test dataset
dataset = pd.DataFrame({'Pred':predictions, 'Act':y_test.values.ravel()})
dataset['Diff'] = abs((1-(dataset['Pred']/dataset['Act']))*100) 
dataset['Diff'].mean()

46.238205569325466

In [153]:
X_test['Predictions'] = pd.DataFrame(predictions)
X_test['Actual'] = y_test['h']
X_test.head(5)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Reg,Ref,Rego,Refo,Sug,Suf,Wef,Wefo,Weg,Wego,...,Bd,Pef,Peg,heat flux,diameter,quality,mass velocity,filename,Predictions,Actual
204,25471.307840,1196.529721,43577.943268,2879.734587,1.711479e+06,228838.033294,6.256317,36.239043,379.079941,1109.588058,...,1.161392,3996.830758,22609.763296,7.5000,0.868,0.5845,600.0,10,9119.755091,14271.8
122,37683.367712,185.615805,44354.246366,1234.147640,2.960745e+06,288480.212681,0.119430,5.279809,479.621317,664.460961,...,5.216088,1069.691763,25405.695275,2.5042,2.320,0.8496,199.9,33,9509.927746,5080
2325,6257.390343,10573.732180,284426.833763,10811.587096,3.457504e+06,369538.016447,302.550231,316.314994,11.324624,23397.983415,...,6.326084,52410.371366,4704.666620,10.1700,2.200,0.0220,1403.0,29,12173.460396,8940
244,13474.795015,179.855482,17776.774427,743.204470,8.665057e+05,91124.432757,0.354987,6.061523,209.542880,364.698937,...,0.359024,858.011533,10154.674367,8.9000,0.510,0.7580,386.0,8,5242.266177,13500
55,8966.714083,569.304949,17581.792319,1161.846834,2.030903e+06,271547.435821,1.193560,4.971095,39.589267,152.207870,...,1.635361,1901.679073,7959.358986,3.1300,1.030,0.5100,204.0,29,10416.490892,5480


In [145]:
dataset = X_test.merge(df_Author_map, how='left', left_on=['filename'], right_on=['index_y'])
dataset.head(3)

,Reg,Ref,Rego,Refo,Sug,Suf,Wef,Wefo,Weg,Wego,...,diameter,quality,mass velocity,filename,Predictions,Actual,index_x,filename_x,index_y,filename_y
0,25471.307840,1196.529721,43577.943268,2879.734587,1.711479e+06,228838.033294,6.256317,36.239043,379.079941,1109.588058,...,0.868,0.5845,600.0,10,9119.755091,14271.8,Daniel & Ribatski (2017).xls,685,10,685
1,37683.367712,185.615805,44354.246366,1234.147640,2.960745e+06,288480.212681,0.119430,5.279809,479.621317,664.460961,...,2.320,0.8496,199.9,33,9509.927746,5080,Tibirica & Ribatski (2010).xls,130,33,130
2,6257.390343,10573.732180,284426.833763,10811.587096,3.457504e+06,369538.016447,302.550231,316.314994,11.324624,23397.983415,...,2.200,0.0220,1403.0,29,12173.460396,8940,Ong (2010).xls,2504,29,2504


In [147]:
dataset = dataset.drop(['filename','filename_x','filename_y','index_y'],axis=1)
# Write the results of test dataset in excel file
import xlsxwriter as xlsw
xlsfile = 'Test_Results.xlsx'
writer = pd.ExcelWriter(xlsfile, engine='xlsxwriter')

dataset.to_excel(writer, sheet_name='Sheet1',startrow=0, startcol=0, header=True ,index=True)
writer.save()